In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import TweedieRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("../data/InsNova_data_2023_train.csv")
df.head()

,id,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,...,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind,clm,numclaims,claimcst0
0,1,0.77,0.444504,SEDAN,4,M,D,3,petrol,147,...,S,1,weekday,6pm - 12am,6,640.448137,1.0,0,0,0.0
1,2,4.45,0.562183,STNWG,1,M,A,3,petrol,158,...,S,1,weekday,6am - 12pm,12,683.749691,0.0,0,0,0.0
2,3,4.90,0.465244,STNWG,1,F,A,3,petrol,159,...,M,1,weekday,6pm - 12am,6,653.656117,1.0,0,0,0.0
3,4,0.48,0.271039,PANVN,4,M,A,4,petrol,80,...,S,1,weekday,12pm - 6pm,12,642.574671,0.0,0,0,0.0
4,5,0.85,0.141624,SEDAN,4,F,A,5,petrol,126,...,S,0,weekday,6am - 12pm,6,647.175035,0.0,0,0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22619 entries, 0 to 22618
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     22619 non-null  int64  
 1   veh_value              22619 non-null  float64
 2   exposure               22619 non-null  float64
 3   veh_body               22619 non-null  object 
 4   veh_age                22619 non-null  int64  
 5   gender                 22619 non-null  object 
 6   area                   22619 non-null  object 
 7   agecat                 22619 non-null  int64  
 8   engine_type            22619 non-null  object 
 9   max_power              22619 non-null  int64  
 10  driving_history_score  22619 non-null  float64
 11  veh_color              22619 non-null  object 
 12  marital_status         22619 non-null  object 
 13  e_bill                 22619 non-null  int64  
 14  time_of_week_driven    22619 non-null  object 
 15  ti

In [4]:
df["high_education_ind"] = df["high_education_ind"].astype("object")

In [5]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=["object"]).columns

In [6]:
# Perform one-hot encoding for categorical variables
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [7]:
# Separate predictors and target
X = df.drop(columns=["id", "clm", "numclaims", "claimcst0"], axis=1)
y = df["claimcst0"]

In [8]:
# Split the data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [10]:
model = linear_model.TweedieRegressor()

In [42]:
# Define the parameter distribution for random search
param_dist = {
    "power": np.arange(0, 2.0, 0.01),  # Adjust as needed
    "alpha": np.arange(0, 1.0, 0.01),  # Adjust as needed
    "max_iter": np.arange(10000, 50000, 1000),
    "solver": ["lbfgs", "newton-cholesky"],
    "warm_start": [True, False],
}

# Create the random search with cross-validation
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    random_state=42,
)

In [43]:
# Fit the grid search to the data
random_search.fit(X_train_scaled, y_train)

# Print the best parameters and corresponding score
print("Best Parameters: ", random_search.best_params_)
print("Best Negative Mean Squared Error: ", random_search.best_score_)

/Users/jiayizhou/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights


Best Parameters:  {'warm_start': False, 'solver': 'newton-cholesky', 'power': 1.85, 'max_iter': 45000, 'alpha': 0.52}
Best Negative Mean Squared Error:  -1533056.752200898


/Users/jiayizhou/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


## Test Prediction

In [13]:
test = pd.read_csv("../data/InsNova_data_2023_vh.csv")

In [14]:
test["high_education_ind"] = test["high_education_ind"].astype("object")

In [15]:
# Identify categorical columns
categorical_cols = test.select_dtypes(include=["object"]).columns
# Perform one-hot encoding for categorical variables
test = pd.get_dummies(test, columns=categorical_cols, drop_first=True)
test = test.rename(columns={"high_education_ind_1": "high_education_ind_1.0"})
# Separate predictors and target
X_test = test.drop(columns=["id"], axis=1)

In [37]:
# Optionally, you can also evaluate the model on the test set
best_model = random_search.best_estimator_
X_test_scaled = scaler.transform(X_test)
y_pred = best_model.predict(X_test_scaled)
# Clip the predictions to be non-negative
y_pred = np.clip(y_pred, a_min=0, a_max=None)

In [38]:
test["Predict"] = y_pred

In [39]:
test["Predict"].describe()

count    22620.000000
mean       156.594343
std         78.015021
min          3.605516
25%        100.988801
50%        139.148913
75%        193.956057
max        891.222313
Name: Predict, dtype: float64

In [26]:
submission = test[["id", "Predict"]]
submission

,id,Predict
0,1,113.342171
1,2,81.703972
2,3,46.781379
3,4,212.705788
4,5,83.575761
...,...,...
22615,22616,151.930237
22616,22617,151.179687
22617,22618,256.088261
22618,22619,232.978892


In [20]:
submission.to_csv("../output/Tweedie_random_submission.csv", index=False)